In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import xarray as xr
import mplotutils as mpu
import yaml


In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
    
dir_scripts   = f'{path_main}Scripts/'
dir_data_Ymax = f'{path_main}Data/EURO-CORDEX/Yearmax/'
dir_fig       = f'{path_main}Figures/Paper_v2/'


## Prepare variables and parameters

In [ ]:
# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#List of cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade',
          'Zagreb', 'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam',
          'London', 'Dublin', 'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk','Vilnius', 
          'Riga', 'Moscow', 'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

# Load short city names
fname_city_short = dir_scripts + 'City_names_short.yml'
with open(fname_city_short, 'r') as file:
    city_names_short = yaml.safe_load(file)

cities_short = [city_names_short[city] for city in cities]

## Plot

In [ ]:
#Define variable
variab = 'TXx'

#Read data
data = xr.open_dataset(dir_data_Ymax + variab + '_all_models_yearmax.nc')

#Create figure
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ccrs.RotatedPole(pole_latitude=39.25, pole_longitude=-162)))

#Add coastlines and set extent
ax.coastlines(resolution='50m', linewidth=0.75, color='k')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='#bdbdbd')
ax.set_extent([-8, 39, 33, 65], crs=ccrs.PlateCarree())

#Select data
val = data.median('model')[variab]

#Plot
h1 = ax.pcolormesh(data.lon, data.lat, val, transform=ccrs.PlateCarree(), vmin=0, vmax=5, cmap='Reds')

#Colorbar
cbar = mpu.colorbar(h1, ax, orientation='horizontal', extend='max', size=0.06, shrink=0.10, pad=0.06)
cbar.ax.tick_params(labelsize=22)
cbar.set_label('$\Delta$' + variab + ' [°C]', fontsize=26, labelpad=15)\

#Loop over cities
for i0, city in enumerate(cities):
    
    #Get coordinates
    coord = city_coords[city]
    
    #Write number
    ax.text(coord[1], coord[0], str(i0+1), transform=ccrs.PlateCarree(),
            color='k', fontweight='bold', fontsize=12, ha='center', va='center',
            bbox=dict(facecolor='#efedf5', edgecolor='none', alpha=0.8, boxstyle="round,pad=0.4, rounding_size=0.5"))

#Add axis and position it
pos = ax.get_position()
x0  = pos.x0 + pos.x1 - 0.08
xd  = 0.35
ax2 = fig.add_axes([x0, pos.y0 - 0.05, xd, pos.y1 - 0.1])

#Loop over cities
for i0, (city_long, city_short) in enumerate(zip(cities, cities_short)):
    
    if city_long=='NizhnyNovgorod':     city_out = 'Nizhny Novgorod'
    elif city_long=='SaintPetersburg':  city_out = 'Saint Petersburg'
    else:                          city_out = city_long
        
        
    if len(city_long)>4:
        add_short = ' (' + city_short + ')'
    else:
        add_short = ''
        
    #Write city names
    if i0<18:  x_txt = 0
    else:      x_txt = 4.3
    ax2.text(x_txt, len(cities)/2-np.mod(i0, 18), str(i0+1) + ': ' + city_out + add_short, fontsize=18)
    
#Set limits and disable frame
ax2.set_ylim([-2, len(cities)/2 + 3])
ax2.set_xlim([0, 5])
ax2.axis('off')

#Save in file
fig.savefig(dir_fig + 'Fig1_Cities_Europe.png', dpi=200, bbox_inches='tight')


## Additional plot

In [ ]:
#Create figures
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson()))

#Add coastlines and borders
ax.coastlines(resolution='50m', linewidth=0.75, color='#737373', zorder=0)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='#bdbdbd', zorder=0)

#Loop over cities
for i0, city in enumerate(cities):
    
    #Get coordinates
    coord = city_coords[city]
    
    #Write number
    ax.text(coord[1], coord[0], str(i0+1), transform=ccrs.PlateCarree(),
            color='#4a1486', fontweight='bold', fontsize=12, ha='center', va='center',
            bbox=dict(facecolor='#efedf5', edgecolor='none', alpha=0.8, boxstyle="round,pad=0.4, rounding_size=0.5"))
    
#Set extent of map
ax.set_extent([-10.5, 46, 33, 70])    
# ax.set_extent([-10, 35, 30, 70])    

#Add axis and position it
pos = ax.get_position()
x0  = pos.x0 + pos.x1 - 0.08
xd  = 0.35
ax2 = fig.add_axes([x0, pos.y0 , xd, pos.y1 - 0.2])

#Loop over cities
for i0, city in enumerate(cities):
    
    if city=='NizhnyNovgorod':     city_out = 'Nizhny Novgorod'
    elif city=='SaintPetersburg':  city_out = 'Saint Petersburg'
    else:                          city_out = city
    
    #Write city names
    if i0<18:  x_txt = 0
    else:      x_txt = 3
    ax2.text(x_txt, len(cities)/2-np.mod(i0, 18), str(i0+1) + ': ' + city_out, fontsize=15)
    
#Set limits and disable frame
ax2.set_ylim([-2, len(cities)/2 + 3])
ax2.set_xlim([0, 5])
ax2.axis('off')

# #Save in file
# fig.savefig(dir_fig + 'FigS_Cities_Europe.png', dpi=200, bbox_inches='tight')
